<a href="https://colab.research.google.com/github/janvierk/MRP/blob/main/MRP_Afriberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/MRP')

In [ ]:
#installing requitred libries and supressing  the print-out
#!pip install requests==2.31.0 --upgrade -q > NUL
!pip install accelerate -U -q > NUL
!pip install transformers -q > NUL
!pip install torch -q > NUL
#!pip install -U transformers -q >NULip install datasets -q > NUL
!pip install tensorflow -q > NUL
!pip install datasets -q > NUL
!pip install evaluate -q > NUL

In [ ]:
#Required packages and libries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers  import Trainer
from transformers import TrainingArguments
from transformers import XLMRobertaTokenizer
from transformers import XLMRobertaForSequenceClassification
from transformers import MT5ForConditionalGeneration
from transformers import T5Tokenizer
import seaborn as sn
import matplotlib
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import AutoModelForSequenceClassification
import transformers as tf
import torch
from datasets import load_dataset
import datasets
import  evaluate


In [ ]:
#swahili_datasets
swahili_testing = pd.read_csv("swahili_testing.csv")
swahili_train = pd.read_csv("swahili_train.csv")
swahili_validation = pd.read_csv("swahili_validation.csv")

In [ ]:
#afriberta model and tokenizer
from transformers import AutoTokenizer, AutoModelForTokenClassification
afriberta_model = AutoModelForSequenceClassification.from_pretrained("castorini/afriberta_large", num_labels = swahili_train["label"].nunique())
afriberta_tokenizer = AutoTokenizer.from_pretrained("castorini/afriberta_large", use_fast = False)
afriberta_tokenizer.model_max_length = 512

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/503M [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at castorini/afriberta_large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/257 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.55M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
# Tokenizing function the dataset
def tokenize_Afriberta(text):
    return afriberta_tokenizer(text,
                     padding="max_length",
                     truncation=True)


In [ ]:
#tokenized swahili data  for Afriberta
tokenized_Afriberta_train=  swahili_train["text"].apply(tokenize_Afriberta)
tokenized_Afriberta_validation= swahili_validation["text"].apply(tokenize_Afriberta)
tokenized_Afriberta_testing = swahili_testing["text"].apply(tokenize_Afriberta)

In [ ]:
#checking if the text has been tokenized accordingly
for i in range(3):
    print(f"Tokenized entry {i+1}:")
    print(f"Input IDs: {tokenized_Afriberta_train.iloc[i]['input_ids']}")
    print(f"Attention Mask: {tokenized_Afriberta_train.iloc[i]['attention_mask']}")
    print("\n")

Tokenized entry 1:
Input IDs: [0, 24328, 417, 4473, 10283, 5171, 36130, 3300, 485, 19175, 14989, 485, 49778, 10338, 1437, 5846, 266, 758, 3535, 2601, 538, 8616, 268, 8426, 297, 9375, 442, 1615, 40283, 40205, 998, 1185, 43517, 1859, 268, 6921, 1040, 1620, 266, 4128, 268, 3508, 268, 5208, 268, 1822, 268, 261, 264, 271, 264, 538, 5448, 272, 1867, 262, 942, 263, 40170, 330, 858, 272, 2543, 34092, 1543, 262, 13258, 5277, 1694, 262, 37954, 17133, 6693, 3633, 11306, 1040, 304, 11845, 298, 5131, 1428, 272, 23915, 272, 5452, 2856, 5183, 1060, 1615, 272, 61151, 10991, 266, 14609, 2087, 261, 276, 265, 328, 3508, 261, 267, 265, 24223, 409, 20241, 2128, 268, 17813, 304, 6181, 501, 263, 2963, 1357, 27808, 40624, 1115, 11306, 1746, 6693, 1794, 437, 24956, 2739, 4019, 397, 8106, 266, 25230, 353, 6181, 621, 304, 501, 3178, 621, 18726, 2393, 4883, 2087, 1706, 266, 8160, 542, 7172, 3182, 2024, 9309, 266, 14609, 2087, 261, 276, 265, 2811, 6871, 588, 277, 18642, 23915, 1751, 12567, 2167, 1694, 5277, 263, 1

In [ ]:
#getting labels for swahili datasets
swahili_train_labels = swahili_train["label"]
swahili_validation_labels = swahili_validation["label"]
swahili_testing_labels = swahili_testing["label"]

In [ ]:
#extracting input ids and attention mask for swahili training
input_ids_Afriberta_train= [item["input_ids"] for item in tokenized_Afriberta_train]
attention_mask_Afriberta_train = [item["attention_mask"] for item in tokenized_Afriberta_train]
Afriberta_encodings_train= {"input_ids": input_ids_Afriberta_train,
             "attention_mask": attention_mask_Afriberta_train}

In [ ]:
#extracting input ids and attention mask for swahili validation
input_ids_Afriberta_validation = [item["input_ids"] for item in tokenized_Afriberta_validation]
attention_mask_Afriberta_validation = [item["attention_mask"] for item in tokenized_Afriberta_validation]
Afriberta_encodings_validation = {"input_ids": input_ids_Afriberta_validation,
             "attention_mask": attention_mask_Afriberta_validation}

In [ ]:
#extracting input ids and attention mask for swahili testing
input_ids_Afriberta_testing= [item["input_ids"] for item in tokenized_Afriberta_testing]
attention_mask_Afriberta_testing = [item["attention_mask"] for item in tokenized_Afriberta_testing]
Afriberta_encodings_testing = {"input_ids": input_ids_Afriberta_testing,
             "attention_mask": attention_mask_Afriberta_testing}

In [ ]:
#creating tensor data to feed into the model
import torch
from torch.utils.data import Dataset, DataLoader

class TextDataset (Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item["labels"] = torch.tensor(self.labels[idx])
    return item
  def __len__(self):
    return len(self.labels)

In [ ]:
#creating tensor data format to feed the model
Afriberta_ready_train = TextDataset(Afriberta_encodings_train, swahili_train_labels)
Afriberta_ready_validation= TextDataset(Afriberta_encodings_validation, swahili_validation_labels)
Afriberta_ready_testing = TextDataset(Afriberta_encodings_testing, swahili_testing_labels)

In [ ]:
#computing the evaluation metrics
"""
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    accuracy = accuracy_metric.compute(predictions=preds, references=labels)
    f1 = f1_metric.compute(predictions=preds, references=labels, average='weighted')
    precision = precision_metric.compute(predictions=preds, references=labels, average='weighted')
    recall = recall_metric.compute(predictions=preds, references=labels, average='weighted')

    return {
        'accuracy': accuracy['accuracy'],
        'f1': f1['f1'],
        'precision': precision['precision'],
        'recall': recall['recall']
    }"""

In [ ]:
from transformers import EarlyStoppingCallback
from sklearn.metrics import confusion_matrix

In [ ]:
#computing the evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    accuracy = accuracy_metric.compute(predictions=preds, references=labels)
    f1 = f1_metric.compute(predictions=preds, references=labels, average='weighted')
    precision = precision_metric.compute(predictions=preds, references=labels, average='weighted')
    recall = recall_metric.compute(predictions=preds, references=labels, average='weighted')
    conf_matrix = confusion_matrix(labels, preds)
    conf_matrix_list = conf_matrix.tolist()

    return {
        'accuracy': accuracy['accuracy'],
        'f1': f1['f1'],
        'precision': precision['precision'],
        'recall': recall['recall'],
        'conf_matrix': conf_matrix_list
    }

In [ ]:
import torch
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import torch.nn as nn
# Calculate class weights using sklearn's compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(swahili_train_labels), y=swahili_train_labels)

# Convert class weights to a tensor for PyTorch
class_weights = torch.tensor(class_weights, dtype=torch.float)
class CustomTrainer(Trainer):
    def __init__(self, class_weights, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights.to(self.args.device)  # Move weights to the appropriate device

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Define the loss function with class weights
        loss_fct = nn.CrossEntropyLoss(weight=self.class_weights)
        loss = loss_fct(logits, labels)


In [ ]:
#training argument for swahbert
Afriberta_args = TrainingArguments(
    output_dir="./Afriberta_results1",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=200,
    weight_decay=0.01,
    logging_dir="./Afriberta_logs1",
    logging_steps=20,
    load_best_model_at_end=True,
    eval_strategy="epoch",
    save_strategy="epoch"
)

In [ ]:
#settting trainer for Afirberta
Afriberta_trainer = CustomTrainer(
    model=afriberta_model,
    args=Afriberta_args,
    train_dataset=Afriberta_ready_train,
    eval_dataset=Afriberta_ready_validation,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=1)],
    class_weights=class_weights)

In [ ]:
#training
Afriberta_trainer.train()

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

In [ ]:
#evaluate
Afriberta_evaluate = Afriberta_trainer.evaluate(eval_dataset=Afriberta_ready_testing)

In [ ]:
print(Afriberta_evaluate)

{'eval_loss': 0.23123207688331604, 'eval_accuracy': 0.9355220849551532, 'eval_f1': 0.9325478872313651, 'eval_precision': 0.9333639638935057, 'eval_recall': 0.9355220849551532, 'eval_runtime': 34.358, 'eval_samples_per_second': 171.983, 'eval_steps_per_second': 10.769, 'epoch': 3.0}
